In [5]:
import transformers
import json
import sys
import os
import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'
config_file_location = '../data/config.json'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

def update_config(checkpoint, file_location = '../data/config.json'):
    config_file = open(file_location, 'r')
    config = json.load(config_file)
    print(config)
    config["_name_or_path"] = checkpoint
    config_file = open(file_location, 'w')
    json.dump(config, config_file, indent= 2)
    config_file.close()

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [6]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val, config_file = '../data/'):
    !python run_qa_alt.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/full_squad_covidQA/ \
      --config_name {config_file} \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 40\
      --per_device_eval_batch_size 40\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 2e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter @ukp/roberta-base_qa_squad1_houlsby

In [7]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-a396986d6b3a2375
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-a396986d6b3a2375\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-8fdbe041288a2f4d
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [ ]:
k_fold = 5
for i in range(k_fold):
    covid_fold = covid_qa.shard(k_fold, i)

    covid_test = covid_fold.shard(2, 0)
    covid_val = covid_fold.shard(2, 1)
    covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

    #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

    checkpoint = 'roberta-base'
    cur_dir = '../models/adapter_baseline/split_' + str(i)

    #log_file = open(cur_dir + "/log_file.txt","w+")
    #sys.stdout = log_file

    squad_qa.shuffle()
    bio_qa.shuffle()
    full_dataset = datasets.concatenate_datasets([bio_qa, squad_qa, covid_train])
    make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)
    run_gradual_ft(cur_dir, checkpoint, covid_val)


Loading cached shuffled indices for dataset at C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16\cache-27e3cdcbb2268e0f.arrow
Loading cached shuffled indices for dataset at C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e\cache-446f7f9c69f4ae06.arrow
